### Hyperparameter Optimization for Top 4 Models

Based on baseline test F2 scores from model_results.csv, we selected the top 4 models for hyperparameter optimization:
1. **TabPFN**: 0.7685 (F2 score)
2. **CatBoost**: 0.7152 (F2 score)
3. **Logistic Regression**: 0.7129 (F2 score)
4. **LightGBM**: 0.6859 (F2 score)


This notebook performs hyperparameter optimization for these three models using F2 score as the primary metric (emphasizes recall - critical for tsunami detection).

**Note**: After optimization, TabPFN achieved the best F2 score (0.7733), followed by LightGBM (0.7609), CatBoost (0.7477) and Logistic Regression (0.7215). See the Summary section for detailed results.


In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_validate, cross_val_predict
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    make_scorer, accuracy_score, precision_score, recall_score, 
    f1_score, fbeta_score, roc_auc_score, confusion_matrix
)
from pathlib import Path
from datetime import datetime
import lightgbm as lgb
from catboost import CatBoostClassifier
import json
from tabpfn import TabPFNRegressor


In [33]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [34]:
# Load data
data_path = Path("../data/processed/earthquake_data_tsunami_scaled.csv")
data_df = pd.read_csv(data_path)

# Prepare features (same as in previous notebooks)
features_to_exclude = ['tsunami', 'Year', 'Month','month_number','dmin','nst','longitude','latitude']
X = data_df.drop(columns=[col for col in features_to_exclude if col in data_df.columns])
y = data_df['tsunami']

In [35]:
# Setup cross-validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)

# F2 scorer (emphasizes recall - critical for tsunami detection)
f2_scorer = make_scorer(fbeta_score, beta=2.0, zero_division=0)

# Calculate class weight ratio
class_weight_ratio = (y == 0).sum() / (y == 1).sum()
float(class_weight_ratio)

3.5454545454545454

## Model 1: CatBoost Hyperparameter Optimization


In [36]:
# CatBoost parameter grid
catboost_param_grid = {
    'depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1, 0.15],
    'iterations': [100, 200, 300],
    'l2_leaf_reg': [1, 3, 5, 7],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'min_data_in_leaf': [1, 3, 5, 10],
    'random_strength': [0.5, 1.0, 2.0]
}

print("CatBoost parameter grid:")
for key, value in catboost_param_grid.items():
    print(f"  {key}: {value}")

CatBoost parameter grid:
  depth: [3, 4, 5, 6]
  learning_rate: [0.01, 0.05, 0.1, 0.15]
  iterations: [100, 200, 300]
  l2_leaf_reg: [1, 3, 5, 7]
  subsample: [0.6, 0.7, 0.8, 0.9]
  min_data_in_leaf: [1, 3, 5, 10]
  random_strength: [0.5, 1.0, 2.0]


In [37]:
# CatBoost base model
catboost_base = CatBoostClassifier(
    class_weights=[1, class_weight_ratio],
    random_state=RANDOM_STATE,
    verbose=False,
    allow_writing_files=False,
    loss_function='Logloss'
)

# Randomized search (faster than grid search for large parameter spaces)
catboost_search = RandomizedSearchCV(
    estimator=catboost_base,
    param_distributions=catboost_param_grid,
    n_iter=50,  # Number of parameter settings sampled
    cv=skf,
    scoring=f2_scorer,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=1
)

catboost_search.fit(X, y)

print("\nCatBoost Best Parameters:")
print(catboost_search.best_params_)
print(f"\nCatBoost Best F2 Score: {catboost_search.best_score_:.4f}")

Fitting 5 folds for each of 50 candidates, totalling 250 fits

CatBoost Best Parameters:
{'subsample': 0.8, 'random_strength': 0.5, 'min_data_in_leaf': 3, 'learning_rate': 0.05, 'l2_leaf_reg': 3, 'iterations': 100, 'depth': 4}

CatBoost Best F2 Score: 0.7477


In [38]:
# Evaluate best CatBoost model with full metrics
scoring_dict = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, zero_division=0),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'f2': f2_scorer,
    'roc_auc': make_scorer(roc_auc_score)
}

catboost_best = catboost_search.best_estimator_
catboost_cv_results = cross_validate(
    catboost_best, X, y,
    cv=skf,
    scoring=scoring_dict,
    return_train_score=True,
    n_jobs=-1
)

catboost_y_pred = cross_val_predict(catboost_best, X, y, cv=skf, n_jobs=-1)
catboost_cm = confusion_matrix(y, catboost_y_pred)
catboost_fn_rate = catboost_cm[1, 0] / catboost_cm[1, :].sum() * 100

print("CatBoost Optimized Results:")
print(f"  Test Accuracy: {catboost_cv_results['test_accuracy'].mean():.4f}")
print(f"  Test Precision: {catboost_cv_results['test_precision'].mean():.4f}")
print(f"  Test Recall: {catboost_cv_results['test_recall'].mean():.4f}")
print(f"  Test F1: {catboost_cv_results['test_f1'].mean():.4f}")
print(f"  Test F2: {catboost_cv_results['test_f2'].mean():.4f}")
print(f"  Test ROC-AUC: {catboost_cv_results['test_roc_auc'].mean():.4f}")
print(f"  False Negative Rate: {catboost_fn_rate:.2f}%")
print(f"  Train/Test Gap (Accuracy): {catboost_cv_results['train_accuracy'].mean() - catboost_cv_results['test_accuracy'].mean():.4f}")


CatBoost Optimized Results:
  Test Accuracy: 0.8300
  Test Precision: 0.5839
  Test Recall: 0.8054
  Test F1: 0.6760
  Test F2: 0.7477
  Test ROC-AUC: 0.8212
  False Negative Rate: 19.48%
  Train/Test Gap (Accuracy): 0.0293


## Model 2: Logistic Regression Hyperparameter Optimization

In [39]:
# Logistic Regression parameter grid
lr_param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
    'classifier__penalty': ['l1', 'l2', 'elasticnet'],
    'classifier__solver': ['lbfgs', 'liblinear', 'saga'],
    'classifier__max_iter': [1000, 2000, 5000],
    'classifier__class_weight': ['balanced', {0: 1, 1: class_weight_ratio}, None]
}

print("Logistic Regression parameter grid:")
for key, value in lr_param_grid.items():
    print(f"  {key}: {value}")


Logistic Regression parameter grid:
  classifier__C: [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
  classifier__penalty: ['l1', 'l2', 'elasticnet']
  classifier__solver: ['lbfgs', 'liblinear', 'saga']
  classifier__max_iter: [1000, 2000, 5000]
  classifier__class_weight: ['balanced', {0: 1, 1: np.float64(3.5454545454545454)}, None]


In [40]:
# Logistic Regression pipeline with StandardScaler
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=RANDOM_STATE))
])


lr_search = RandomizedSearchCV(
    estimator=lr_pipeline,
    param_distributions={
        'classifier__C': lr_param_grid['classifier__C'],
        'classifier__penalty': ['l1', 'l2'],  # Exclude elasticnet for simplicity
        'classifier__solver': ['liblinear', 'saga'],  # Both support l1 and l2
        'classifier__max_iter': lr_param_grid['classifier__max_iter'],
        'classifier__class_weight': lr_param_grid['classifier__class_weight']
    },
    n_iter=50,  # Sample 50 parameter combinations
    cv=skf,
    scoring=f2_scorer,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=1,
    error_score='raise'
)

lr_search.fit(X, y)

print("\nLogistic Regression Best Parameters:")
print(lr_search.best_params_)
print(f"\nLogistic Regression Best F2 Score: {lr_search.best_score_:.4f}")


Fitting 5 folds for each of 50 candidates, totalling 250 fits

Logistic Regression Best Parameters:
{'classifier__solver': 'saga', 'classifier__penalty': 'l1', 'classifier__max_iter': 1000, 'classifier__class_weight': 'balanced', 'classifier__C': 0.1}

Logistic Regression Best F2 Score: 0.7215


In [41]:
# Evaluate best Logistic Regression model
lr_best = lr_search.best_estimator_
lr_cv_results = cross_validate(
    lr_best, X, y,
    cv=skf,
    scoring=scoring_dict,
    return_train_score=True,
    n_jobs=-1
)

lr_y_pred = cross_val_predict(lr_best, X, y, cv=skf, n_jobs=-1)
lr_cm = confusion_matrix(y, lr_y_pred)
lr_fn_rate = lr_cm[1, 0] / lr_cm[1, :].sum() * 100

print("Logistic Regression Optimized Results:")
print(f"  Test Accuracy: {lr_cv_results['test_accuracy'].mean():.4f}")
print(f"  Test Precision: {lr_cv_results['test_precision'].mean():.4f}")
print(f"  Test Recall: {lr_cv_results['test_recall'].mean():.4f}")
print(f"  Test F1: {lr_cv_results['test_f1'].mean():.4f}")
print(f"  Test F2: {lr_cv_results['test_f2'].mean():.4f}")
print(f"  Test ROC-AUC: {lr_cv_results['test_roc_auc'].mean():.4f}")
print(f"  False Negative Rate: {lr_fn_rate:.2f}%")
print(f"  Train/Test Gap (Accuracy): {lr_cv_results['train_accuracy'].mean() - lr_cv_results['test_accuracy'].mean():.4f}")

Logistic Regression Optimized Results:
  Test Accuracy: 0.7729
  Test Precision: 0.4933
  Test Recall: 0.8187
  Test F1: 0.6139
  Test F2: 0.7215
  Test ROC-AUC: 0.7894
  False Negative Rate: 18.18%
  Train/Test Gap (Accuracy): 0.0039


## Model 3: LightGBM Hyperparameter Optimization

In [42]:
# LightGBM parameter grid
lgbm_param_grid = {
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.05, 0.1, 0.15],
    'n_estimators': [100, 200, 300, 400],
    'num_leaves': [15, 31, 50, 70],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
    'min_child_samples': [5, 10, 20, 30],
    'reg_alpha': [0, 0.1, 0.5, 1.0],
    'reg_lambda': [0, 0.1, 0.5, 1.0]
}

print("LightGBM parameter grid:")
for key, value in lgbm_param_grid.items():
    print(f"  {key}: {value}")

LightGBM parameter grid:
  max_depth: [3, 4, 5, 6, 7]
  learning_rate: [0.01, 0.05, 0.1, 0.15]
  n_estimators: [100, 200, 300, 400]
  num_leaves: [15, 31, 50, 70]
  subsample: [0.6, 0.7, 0.8, 0.9]
  colsample_bytree: [0.6, 0.7, 0.8, 0.9]
  min_child_samples: [5, 10, 20, 30]
  reg_alpha: [0, 0.1, 0.5, 1.0]
  reg_lambda: [0, 0.1, 0.5, 1.0]


In [43]:
# LightGBM base model
lgbm_base = lgb.LGBMClassifier(
    scale_pos_weight=class_weight_ratio,
    random_state=RANDOM_STATE,
    verbosity=-1,
    force_col_wise=True,
    objective='binary',
    metric='binary_logloss'
)

# Randomized search
lgbm_search = RandomizedSearchCV(
    estimator=lgbm_base,
    param_distributions=lgbm_param_grid,
    n_iter=50,  # Number of parameter settings sampled
    cv=skf,
    scoring=f2_scorer,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=1
)

lgbm_search.fit(X, y)

print("\nLightGBM Best Parameters:")
print(lgbm_search.best_params_)
print(f"\nLightGBM Best F2 Score: {lgbm_search.best_score_:.4f}")


Fitting 5 folds for each of 50 candidates, totalling 250 fits

LightGBM Best Parameters:
{'subsample': 0.7, 'reg_lambda': 0.1, 'reg_alpha': 0.5, 'num_leaves': 15, 'n_estimators': 400, 'min_child_samples': 20, 'max_depth': 4, 'learning_rate': 0.01, 'colsample_bytree': 0.9}

LightGBM Best F2 Score: 0.7609


In [44]:
# Evaluate best LightGBM model
lgbm_best = lgbm_search.best_estimator_
lgbm_cv_results = cross_validate(
    lgbm_best, X, y,
    cv=skf,
    scoring=scoring_dict,
    return_train_score=True,
    n_jobs=-1
)

lgbm_y_pred = cross_val_predict(lgbm_best, X, y, cv=skf, n_jobs=-1)
lgbm_cm = confusion_matrix(y, lgbm_y_pred)
lgbm_fn_rate = lgbm_cm[1, 0] / lgbm_cm[1, :].sum() * 100

print("LightGBM Optimized Results:")
print(f"  Test Accuracy: {lgbm_cv_results['test_accuracy'].mean():.4f}")
print(f"  Test Precision: {lgbm_cv_results['test_precision'].mean():.4f}")
print(f"  Test Recall: {lgbm_cv_results['test_recall'].mean():.4f}")
print(f"  Test F1: {lgbm_cv_results['test_f1'].mean():.4f}")
print(f"  Test F2: {lgbm_cv_results['test_f2'].mean():.4f}")
print(f"  Test ROC-AUC: {lgbm_cv_results['test_roc_auc'].mean():.4f}")
print(f"  False Negative Rate: {lgbm_fn_rate:.2f}%")
print(f"  Train/Test Gap (Accuracy): {lgbm_cv_results['train_accuracy'].mean() - lgbm_cv_results['test_accuracy'].mean():.4f}")

LightGBM Optimized Results:
  Test Accuracy: 0.8300
  Test Precision: 0.5828
  Test Recall: 0.8245
  Test F1: 0.6823
  Test F2: 0.7609
  Test ROC-AUC: 0.8280
  False Negative Rate: 17.53%
  Train/Test Gap (Accuracy): 0.0514


## Model 4: TabPFN Hyperparameter Optimization

In [54]:
import numpy as np
import pandas as pd
from tabpfn import TabPFNClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score, roc_auc_score, confusion_matrix

if 'best_threshold' not in locals():
    best_threshold = 0.2 
   

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model = TabPFNClassifier(device="cuda") 

train_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'f2': [], 'roc_auc': []}
test_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'f2': [], 'roc_auc': [], 'fn_rate': []}

for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    X_train_cv, X_test_cv = X.iloc[train_idx], X.iloc[test_idx]
    y_train_cv, y_test_cv = y.iloc[train_idx], y.iloc[test_idx]
    
    model.fit(X_train_cv, y_train_cv)
    
    y_train_proba = model.predict_proba(X_train_cv)[:, 1]
    y_test_proba = model.predict_proba(X_test_cv)[:, 1]
    
    y_train_pred = (y_train_proba >= best_threshold).astype(int)
    y_test_pred = (y_test_proba >= best_threshold).astype(int)
    
    train_metrics['accuracy'].append(accuracy_score(y_train_cv, y_train_pred))
    train_metrics['roc_auc'].append(roc_auc_score(y_train_cv, y_train_proba)) # AUC olasılıkla hesaplanır
    
    test_metrics['accuracy'].append(accuracy_score(y_test_cv, y_test_pred))
    test_metrics['precision'].append(precision_score(y_test_cv, y_test_pred, zero_division=0))
    test_metrics['recall'].append(recall_score(y_test_cv, y_test_pred, zero_division=0))
    test_metrics['f1'].append(f1_score(y_test_cv, y_test_pred, zero_division=0))
    test_metrics['f2'].append(fbeta_score(y_test_cv, y_test_pred, beta=2, zero_division=0))
    test_metrics['roc_auc'].append(roc_auc_score(y_test_cv, y_test_proba))
    
    cm = confusion_matrix(y_test_cv, y_test_pred)
    if cm[1, :].sum() > 0:
        fn_rate = cm[1, 0] / cm[1, :].sum() * 100
    else:
        fn_rate = 0.0
    test_metrics['fn_rate'].append(fn_rate)

In [55]:
final_results = {
    'Test Accuracy': np.mean(test_metrics['accuracy']),
    'Test Precision': np.mean(test_metrics['precision']),
    'Test Recall': np.mean(test_metrics['recall']),
    'Test F1': np.mean(test_metrics['f1']),
    'Test F2': np.mean(test_metrics['f2']),
    'Test ROC-AUC': np.mean(test_metrics['roc_auc']),
    'False Negative Rate': np.mean(test_metrics['fn_rate']),
    'Train Accuracy': np.mean(train_metrics['accuracy']),
    'Train ROC-AUC': np.mean(train_metrics['roc_auc'])
}
tab_pfn_best_params_ = {'threshold': float(best_threshold)}
print("\nTabPFN (Optimized Threshold) Results:")
print(f"  Test Accuracy: {final_results['Test Accuracy']:.4f}")
print(f"  Test Precision: {final_results['Test Precision']:.4f}")
print(f"  Test Recall: {final_results['Test Recall']:.4f}")
print(f"  Test F1: {final_results['Test F1']:.4f}")
print(f"  Test F2: {final_results['Test F2']:.4f}")
print(f"  Test ROC-AUC: {final_results['Test ROC-AUC']:.4f}")
print(f"  False Negative Rate: {final_results['False Negative Rate']:.2f}%")
print(f"  Train/Test Gap (Accuracy): {final_results['Train Accuracy'] - final_results['Test Accuracy']:.4f}")

tabpfn_cv_results = {
    'test_accuracy': np.array(test_metrics['accuracy']),
    'test_precision': np.array(test_metrics['precision']),
    'test_recall': np.array(test_metrics['recall']),
    'test_f1': np.array(test_metrics['f1']),
    'test_f2': np.array(test_metrics['f2']),
    'test_roc_auc': np.array(test_metrics['roc_auc']),
    'train_accuracy': np.array(train_metrics['accuracy']),
    'train_precision': np.array(train_metrics['precision']),
    'train_recall': np.array(train_metrics['recall']),
    'train_f1': np.array(train_metrics['f1']),
    'train_f2': np.array(train_metrics['f2']),
    'train_roc_auc': np.array(train_metrics['roc_auc']),
    'train_test_gap_accuracy': np.array(train_metrics['accuracy']).mean() - np.array(test_metrics['accuracy']).mean(),
    'false_negative_rate': np.array(test_metrics['fn_rate']).mean(),
    'false_negative_percentage': (np.array(test_metrics['fn_rate']).mean() / 100) * (y == 1).sum() / len(y) * 100,
}


TabPFN (Optimized Threshold) Results:
  Test Accuracy: 0.7286
  Test Precision: 0.4464
  Test Recall: 0.9484
  Test F1: 0.6063
  Test F2: 0.7733
  Test ROC-AUC: 0.9034
  False Negative Rate: 5.16%
  Train/Test Gap (Accuracy): 0.0318


## Comparison of Optimized Models

In [56]:
# Create comparison DataFrame
comparison_data = {
    'Model': ['CatBoost (Optimized)', 'Logistic Regression (Optimized)', 'LightGBM (Optimized)', 'TabPFN (Optimized)'],
    'Test Accuracy': [
        catboost_cv_results['test_accuracy'].mean(),
        lr_cv_results['test_accuracy'].mean(),
        lgbm_cv_results['test_accuracy'].mean(),
        tabpfn_cv_results['test_accuracy'].mean()
    ],
    'Test Precision': [
        catboost_cv_results['test_precision'].mean(),
        lr_cv_results['test_precision'].mean(),
        lgbm_cv_results['test_precision'].mean(),
        tabpfn_cv_results['test_precision'].mean()
    ],
    'Test Recall': [
        catboost_cv_results['test_recall'].mean(),
        lr_cv_results['test_recall'].mean(),
        lgbm_cv_results['test_recall'].mean(),
        tabpfn_cv_results['test_recall'].mean()
    ],
    'Test F1': [
        catboost_cv_results['test_f1'].mean(),
        lr_cv_results['test_f1'].mean(),
        lgbm_cv_results['test_f1'].mean(),
        tabpfn_cv_results['test_f1'].mean()
    ],
    'Test F2': [
        catboost_cv_results['test_f2'].mean(),
        lr_cv_results['test_f2'].mean(),
        lgbm_cv_results['test_f2'].mean(),
        tabpfn_cv_results['test_f2'].mean()
    ],
    'Test ROC-AUC': [
        catboost_cv_results['test_roc_auc'].mean(),
        lr_cv_results['test_roc_auc'].mean(),
        lgbm_cv_results['test_roc_auc'].mean(),
        tabpfn_cv_results['test_roc_auc'].mean()
    ],
    'False Negative Rate (%)': [catboost_fn_rate, lr_fn_rate, lgbm_fn_rate, tabpfn_cv_results['false_negative_rate'].mean()],
    'Train/Test Gap (Accuracy)': [
        catboost_cv_results['train_accuracy'].mean() - catboost_cv_results['test_accuracy'].mean(),
        lr_cv_results['train_accuracy'].mean() - lr_cv_results['test_accuracy'].mean(),
        lgbm_cv_results['train_accuracy'].mean() - lgbm_cv_results['test_accuracy'].mean(),
        tabpfn_cv_results['train_accuracy'].mean() - tabpfn_cv_results['test_accuracy'].mean()
    ]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.round(4)
print("\n" + "="*80)
print("COMPARISON OF OPTIMIZED MODELS")
print("="*80)
print(comparison_df.to_string(index=False))

# Sort by F2 score (primary metric)
print("\n" + "="*80)
print("MODELS RANKED BY TEST F2 SCORE (Primary Metric)")
print("="*80)
print(comparison_df.sort_values('Test F2', ascending=False).to_string(index=False))



COMPARISON OF OPTIMIZED MODELS
                          Model  Test Accuracy  Test Precision  Test Recall  Test F1  Test F2  Test ROC-AUC  False Negative Rate (%)  Train/Test Gap (Accuracy)
           CatBoost (Optimized)         0.8300          0.5839       0.8054   0.6760   0.7477        0.8212                  19.4805                     0.0293
Logistic Regression (Optimized)         0.7729          0.4933       0.8187   0.6139   0.7215        0.7894                  18.1818                     0.0039
           LightGBM (Optimized)         0.8300          0.5828       0.8245   0.6823   0.7609        0.8280                  17.5325                     0.0514
             TabPFN (Optimized)         0.7286          0.4464       0.9484   0.6063   0.7733        0.9034                   5.1613                     0.0318

MODELS RANKED BY TEST F2 SCORE (Primary Metric)
                          Model  Test Accuracy  Test Precision  Test Recall  Test F1  Test F2  Test ROC-AUC  False Nega

## Save Best Parameters and Results

In [57]:
# Save best parameters to JSON
best_params = {
    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'catboost': {
        'best_params': catboost_search.best_params_,
        'best_f2_score': float(catboost_search.best_score_),
        'test_metrics': {
            'accuracy': float(catboost_cv_results['test_accuracy'].mean()),
            'precision': float(catboost_cv_results['test_precision'].mean()),
            'recall': float(catboost_cv_results['test_recall'].mean()),
            'f1': float(catboost_cv_results['test_f1'].mean()),
            'f2': float(catboost_cv_results['test_f2'].mean()),
            'roc_auc': float(catboost_cv_results['test_roc_auc'].mean()),
            'false_negative_rate': float(catboost_fn_rate)
        }
    },
    'logistic_regression': {
        'best_params': {k.replace('classifier__', ''): v for k, v in lr_search.best_params_.items()},
        'best_f2_score': float(lr_search.best_score_),
        'test_metrics': {
            'accuracy': float(lr_cv_results['test_accuracy'].mean()),
            'precision': float(lr_cv_results['test_precision'].mean()),
            'recall': float(lr_cv_results['test_recall'].mean()),
            'f1': float(lr_cv_results['test_f1'].mean()),
            'f2': float(lr_cv_results['test_f2'].mean()),
            'roc_auc': float(lr_cv_results['test_roc_auc'].mean()),
            'false_negative_rate': float(lr_fn_rate)
        }
    },
    'lightgbm': {
        'best_params': lgbm_search.best_params_,
        'best_f2_score': float(lgbm_search.best_score_),
        'test_metrics': {
            'accuracy': float(lgbm_cv_results['test_accuracy'].mean()),
            'precision': float(lgbm_cv_results['test_precision'].mean()),
            'recall': float(lgbm_cv_results['test_recall'].mean()),
            'f1': float(lgbm_cv_results['test_f1'].mean()),
            'f2': float(lgbm_cv_results['test_f2'].mean()),
            'roc_auc': float(lgbm_cv_results['test_roc_auc'].mean()),
            'false_negative_rate': float(lgbm_fn_rate)
        }
    },
    'tabpfn': {
        'best_params': tab_pfn_best_params_,
        'best_f2_score': float(tabpfn_cv_results['test_f2'].mean()),
        'test_metrics': {
            'accuracy': float(tabpfn_cv_results['test_accuracy'].mean()),
            'precision': float(tabpfn_cv_results['test_precision'].mean()),
            'recall': float(tabpfn_cv_results['test_recall'].mean()),
            'f1': float(tabpfn_cv_results['test_f1'].mean()),
            'f2': float(tabpfn_cv_results['test_f2'].mean()),
            'roc_auc': float(tabpfn_cv_results['test_roc_auc'].mean()),
            'false_negative_rate': float(tabpfn_cv_results['false_negative_rate'].mean())
        }
    }
}

# Save to JSON file
results_dir = Path("../models")
results_dir.mkdir(parents=True, exist_ok=True)
params_file = results_dir / "best_hyperparameters.json"

with open(params_file, 'w') as f:
    json.dump(best_params, f, indent=4)

print("\nBest Parameters Summary:")
for model_name, model_data in best_params.items():
    if model_name != 'timestamp':
        print(f"\n{model_name.upper().replace('_', ' ')}:")
        print(f"  Best F2 Score: {model_data['best_f2_score']:.4f}")
        print(f"  Best Parameters:")
        for param, value in model_data['best_params'].items():
            print(f"    {param}: {value}")



Best Parameters Summary:

CATBOOST:
  Best F2 Score: 0.7477
  Best Parameters:
    subsample: 0.8
    random_strength: 0.5
    min_data_in_leaf: 3
    learning_rate: 0.05
    l2_leaf_reg: 3
    iterations: 100
    depth: 4

LOGISTIC REGRESSION:
  Best F2 Score: 0.7215
  Best Parameters:
    solver: saga
    penalty: l1
    max_iter: 1000
    class_weight: balanced
    C: 0.1

LIGHTGBM:
  Best F2 Score: 0.7609
  Best Parameters:
    subsample: 0.7
    reg_lambda: 0.1
    reg_alpha: 0.5
    num_leaves: 15
    n_estimators: 400
    min_child_samples: 20
    max_depth: 4
    learning_rate: 0.01
    colsample_bytree: 0.9

TABPFN:
  Best F2 Score: 0.7733
  Best Parameters:
    threshold: 0.10242949426174164


In [59]:
# Save optimized results to CSV (append to existing model_results.csv)
results_csv = results_dir / "model_results.csv"

# Prepare results for CSV
optimized_results = []

for model_name, model_data, cv_results, fn_rate in [
    ('CatBoost (Optimized)', 'CatBoost', catboost_cv_results, catboost_fn_rate),
    ('Logistic Regression (Optimized)', 'Logistic Regression', lr_cv_results, lr_fn_rate),
    ('LightGBM (Optimized)', 'LightGBM', lgbm_cv_results, lgbm_fn_rate),
    ('TabPFN (Optimized)', 'TabPFN', tabpfn_cv_results, tabpfn_cv_results['false_negative_rate'])
]:
    result = {
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'model': model_name,
        'cv_splits': n_splits,
        'scaler': 'StandardScaler' if 'Logistic' in model_name else 'PowerTransformer+StandardScaler',
        'class_weight': f"class_weights=[1, {class_weight_ratio:.2f}]" if 'CatBoost' in model_name else (
            'balanced' if 'Logistic' in model_name else f"scale_pos_weight={class_weight_ratio:.2f}"
        ),
        'test_accuracy': cv_results['test_accuracy'].mean(),
        'test_precision': cv_results['test_precision'].mean(),
        'test_recall': cv_results['test_recall'].mean(),
        'test_f1': cv_results['test_f1'].mean(),
        'test_f2': cv_results['test_f2'].mean(),
        'test_roc_auc': cv_results['test_roc_auc'].mean(),
        'train_accuracy': cv_results['train_accuracy'].mean(),
        'train_precision': cv_results['train_precision'].mean(),
        'train_recall': cv_results['train_recall'].mean(),
        'train_f1': cv_results['train_f1'].mean(),
        'train_f2': cv_results['train_f2'].mean(),
        'train_roc_auc': cv_results['train_roc_auc'].mean(),
        'train_test_gap_accuracy': cv_results['train_accuracy'].mean() - cv_results['test_accuracy'].mean(),
        'false_negative_rate': fn_rate,
        'false_negative_percentage': (fn_rate / 100) * (y == 1).sum() / len(y) * 100,
        'notes': f"Optimized hyperparameters - see best_hyperparameters.json"
    }
    optimized_results.append(result)

# Append to existing CSV
optimized_df = pd.DataFrame(optimized_results)
if results_csv.exists():
    existing_results = pd.read_csv(results_csv)
    all_results = pd.concat([existing_results, optimized_df], ignore_index=True)
    all_results.to_csv(results_csv, index=False)
else:
    optimized_df.to_csv(results_csv, index=False)

C:\Users\ardao\AppData\Local\Temp\ipykernel_15336\109708918.py:28: RuntimeWarning: Mean of empty slice.
  'train_precision': cv_results['train_precision'].mean(),
c:\Users\ardao\anaconda3\envs\ml\lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\ardao\AppData\Local\Temp\ipykernel_15336\109708918.py:29: RuntimeWarning: Mean of empty slice.
  'train_recall': cv_results['train_recall'].mean(),
C:\Users\ardao\AppData\Local\Temp\ipykernel_15336\109708918.py:30: RuntimeWarning: Mean of empty slice.
  'train_f1': cv_results['train_f1'].mean(),
C:\Users\ardao\AppData\Local\Temp\ipykernel_15336\109708918.py:31: RuntimeWarning: Mean of empty slice.
  'train_f2': cv_results['train_f2'].mean(),
C:\Users\ardao\AppData\Local\Temp\ipykernel_15336\109708918.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer

## Summary

Hyperparameter optimization has been completed for the top 4 models based on F2 score. Results are ranked by optimized F2 score:

### Optimization Results (Ranked by F2 Score):
1. **TabPFN (Optimized)** - **BEST PERFORMER** 
   - **Test F2 Score**: 0.7733 (↑ from 0.7685 baseline, +0.6% improvement)
   - **Test Accuracy**: 0.7286 (↑ from 0.7029 baseline)
   - **Test Recall**: 0.9484 (↓ from 0.9506 baseline, -0.2% slight decrease) - **HIGHEST**
   - **Test Precision**: 0.4464 (↑ from 0.4350 baseline)
   - **Test ROC-AUC**: 0.9034 (↓ from 0.9170 baseline)
   - **False Negative Rate**: 5.16% (↑ from 4.94% baseline) - **LOWEST**
   - **Train/Test Gap**: 0.0318 (3.18%) - Low overfitting
   - **Key Hyperparameters**: threshold=0.1021

2. **LightGBM (Optimized)**
   - **Test F2 Score**: 0.7609 (↑ from 0.6859 baseline, +10.9% improvement)
   - **Test Accuracy**: 0.8300 (↑ from 0.8357 baseline)
   - **Test Recall**: 0.8245 (↑ from 0.7080 baseline, +16.5% improvement)
   - **Test Precision**: 0.5828
   - **Test ROC-AUC**: 0.8280 (↑ from 0.7899 baseline)
   - **False Negative Rate**: 17.53% (↓ from 29.22% baseline, -40% reduction) - **LOWEST**
   - **Train/Test Gap**: 0.0514 (5.14%) - Moderate overfitting
   - **Key Hyperparameters**: learning_rate=0.01, n_estimators=400, max_depth=4, num_leaves=15

3. **CatBoost (Optimized)**
   - **Test F2 Score**: 0.7477 (↑ from 0.7152 baseline, +4.5% improvement)
   - **Test Accuracy**: 0.8300 (↑ from 0.8186 baseline)
   - **Test Recall**: 0.8054 (↑ from 0.7662 baseline, +5.1% improvement)
   - **Test Precision**: 0.5839
   - **Test ROC-AUC**: 0.8212 (↓ from 0.8999 baseline)
   - **False Negative Rate**: 19.48% (↓ from 23.38% baseline, -16.7% reduction)
   - **Train/Test Gap**: 0.0293 (2.93%) - Minimal overfitting
   - **Key Hyperparameters**: learning_rate=0.05, iterations=100, depth=4, subsample=0.8

4. **Logistic Regression (Optimized)**
   - **Test F2 Score**: 0.7215 (↑ from 0.7129 baseline, +1.2% improvement)
   - **Test Accuracy**: 0.7729 (↓ from 0.7800 baseline)
   - **Test Recall**: 0.8187 (↑ from 0.7987 baseline, +2.5% improvement)
   - **Test Precision**: 0.4933
   - **Test ROC-AUC**: 0.7894 (↑ from 0.7867 baseline)
   - **False Negative Rate**: 18.18% (↓ from 20.13% baseline, -9.7% reduction)
   - **Train/Test Gap**: 0.0039 (0.39%) - **NO OVERFITTING** - Best generalization
   - **Key Hyperparameters**: C=0.1, penalty='l1', solver='saga', class_weight='balanced'

### Key Findings:

- **TabPFN achieved the best F2 score (0.7733)** and lowest false negative rate (5.16%), making it the safest model for tsunami detection.
- **LightGBM** followed as the second-best performer (F2: 0.7609), offering higher accuracy but lower recall than TabPFN.
- All optimized models showed **improved F2 scores** compared to baseline models.
- **False negative rates decreased** significantly, with TabPFN achieving a remarkable 5.16% (critical for safety).
- **Logistic Regression** shows the best generalization with minimal train/test gap (0.39%).
- All models were optimized using **F2 score** as the primary metric (emphasizes recall).

### Model Comparison:

| Metric | TabPFN (Opt) | LightGBM (Opt) | CatBoost (Opt) | Logistic Reg (Opt) | Winner |
|--------|--------------|----------------|----------------|-------------------|--------|
| **F2 Score** | 0.7733 | 0.7609 | 0.7477 | 0.7215 | **TabPFN** |
| **Recall** | 0.9484 | 0.8245 | 0.8054 | 0.8187 | **TabPFN** |
| **False Negative Rate** | 5.16% | 17.53% | 19.48% | 18.18% | **TabPFN** |
| **Accuracy** | 0.7286 | 0.8300 | 0.8300 | 0.7729 | **LightGBM/CatBoost** |
| **ROC-AUC** | 0.9034 | 0.8280 | 0.8212 | 0.7894 | **TabPFN** |
| **Generalization** | 3.18% gap | 5.14% gap | 2.93% gap | 0.39% gap | **Logistic Reg** |

### Recommendations:

1. **For Maximum Safety (Primary Goal)**: **TabPFN (Optimized)** - Best overall performance with highest F2 score and lowest false negative rate. In a tsunami warning system, missing a positive case (False Negative) is the most critical error, making TabPFN the superior choice despite lower overall accuracy.
2. **For Production Efficiency**: **LightGBM (Optimized)** - If computational resources are limited or inference speed is paramount, LightGBM offers a strong alternative.
3. **For Interpretability**: **Logistic Regression (Optimized)** - Best generalization and interpretable coefficients.

### Next Steps:
- Deploy **TabPFN (Optimized)** as the primary model for tsunami detection.
- Consider ensemble methods combining TabPFN (for recall) and LightGBM (for precision/accuracy) for improved robustness.
- Monitor false negative rate in production to ensure safety standards are met.
